### Домашнее задание

1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4. сделать выводы о том, какая модель справилась с задачей лучше других
5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого). 

p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно). 
Допустим, у нас две модели:

- первая помечает 100 объектов как класс 1, но TP = 90
- вторая помечает 1000 объектов как класс 1, но TP такой же - 90

Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

In [98]:
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from scipy.sparse import hstack
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import xgboost as xgb
import catboost as cb

%matplotlib inline

In [5]:
result={}

In [10]:
df = pd.read_csv('train_case2.csv', ';')
df


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,99993,19240,2,168,76.0,120,80,1,1,1,0,1,0
69996,99995,22601,1,158,126.0,140,90,2,2,0,0,1,1
69997,99996,19066,2,183,105.0,180,90,3,1,0,1,0,1
69998,99998,22431,1,163,72.0,135,80,1,2,0,0,0,1


In [48]:
df_1 = df.drop('id',axis=1)

In [6]:
continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

In [63]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]
        
        
def data_preparation(continuos_cols,cat_cols,base_cols):
    continuos_transformers = []
    cat_transformers = []
    base_transformers = []
    for cont_col in continuos_cols:
        transfomer =  Pipeline([
                    ('selector', NumberSelector(key=cont_col)),
                    ('standard', StandardScaler())
                ])
        continuos_transformers.append((cont_col, transfomer))
    
    for cat_col in cat_cols:
        cat_transformer = Pipeline([
                    ('selector', ColumnSelector(key=cat_col)),
                    ('ohe', OHEEncoder(key=cat_col))
                ])
        cat_transformers.append((cat_col, cat_transformer))
    
    for base_col in base_cols:
        base_transformer = Pipeline([
                    ('selector', NumberSelector(key=base_col))
                ])
        base_transformers.append((base_col, base_transformer))
    
    return continuos_transformers+cat_transformers+base_transformers       


def cros_val(classifier,df,target,scoring = 'roc_auc',cv = 10):
        X_train, X_test, y_train, y_test = train_test_split(df.drop(target, 1), 
                                                    df[target], random_state=0)
        cv_scores = cross_val_score(classifier, X_train, y_train, cv=cv, scoring=scoring)
        cv_score = np.mean(cv_scores)
        cv_score_std = np.std(cv_scores)
        print('CV score is {}+-{}'.format(cv_score, cv_score_std))
        classifier.fit(X_train, y_train)
        try:
            y_score = classifier.predict_proba(X_test)[:, 1]
        except :
            y_score = classifier.predict(X_test)
        precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
        fscore = 2*(precision * recall) / (precision + recall)
        ix = np.argmax(fscore)
        print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))
        
        print('roc_auc на test - ',roc_auc_score(y_test,y_score))
        return cv_score,fscore[ix],precision[ix],recall[ix]


In [37]:
feats = FeatureUnion(data_preparation(continuos_cols,cat_cols,base_cols))
    

In [38]:
classifier = Pipeline([
    ('features',feats),
    ('classifier', LogisticRegression(random_state = 42)),
])

In [51]:
result['logreg']=cros_val(classifier,df_1,'cardio')

CV score is 0.7866749255103078+-0.00652621980218306
Best Threshold=0.386937, F-Score=0.730, Precision=0.647, Recall=0.838
roc_auc на test -  0.7840347790421852


In [101]:
classifier_rf = RandomForestClassifier(n_jobs=-1, random_state=0)

In [102]:
result['rf']=cros_val(classifier_rf,df_1,'cardio')

CV score is 0.7753185140571268+-0.004660108148358007
Best Threshold=0.320000, F-Score=0.721, Precision=0.630, Recall=0.843
roc_auc на test -  0.77236504279131


In [103]:
classifier_lgb = lgb.LGBMClassifier()

In [104]:
result['lgb']=cros_val(classifier_lgb,df_1,'cardio')

CV score is 0.80181205953913+-0.005768187472780772
Best Threshold=0.344379, F-Score=0.739, Precision=0.660, Recall=0.838
roc_auc на test -  0.8009599765405395


In [105]:
classifier_xgb = xgb.XGBRFClassifier()

In [106]:
result['xgb']=cros_val(classifier_xgb,df_1,'cardio')

CV score is 0.7986847196468599+-0.006018803684284639
Best Threshold=0.397684, F-Score=0.737, Precision=0.696, Recall=0.784
roc_auc на test -  0.7994856748382917


In [113]:
classifier_cb = cb.CatBoostClassifier(verbose= False)

In [114]:
result['cb']=cros_val(classifier_cb,df_1,'cardio')

CV score is 0.8020666670542195+-0.006020781158703074
Best Threshold=0.387631, F-Score=0.739, Precision=0.693, Recall=0.791
roc_auc на test -  0.8010239414506202


In [132]:
result_df= pd.DataFrame(index=['logreg','rf','lgb','cb','xgb'],columns=['auc','f_score','precision','recal'])

In [136]:
for i in result_df.index:
    result_df.loc[i] = result[i]

In [138]:
result_df.sort_values('auc')

,auc,f_score,precision,recal
rf,0.775319,0.721195,0.630354,0.842627
logreg,0.786675,0.730323,0.647431,0.837558
xgb,0.798685,0.737013,0.695532,0.783756
lgb,0.801812,0.738585,0.66046,0.837673
cb,0.802067,0.738723,0.693271,0.790553


Как видно регрессия лучше случайного леса, а бустинги лучше регрессии